In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [7]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`              
            layer    coolrg                layer        coolrg
pressure                                                      
0.000312        1 -0.000257                    1 -2.273263e-04
0.000750        2 -0.000197                    2 -1.255252e-04
0.001052        3 -0.000043                    3 -9.686140e-05
0.001476        4  0.000166                    4  4.539884e-05
0.002070        5  0.000441                    5  2.277530e-04
0.002904        6  0.000794                    6  4.535115e-04
0.004074        7  0.001242                    7  7.552338e-04
0.005714        8  0.001805                    8  1.158077e-03
0.008015        9  0.002504                    9  1.660594e-03
0.011243       10  0.003364                   10  2.270661e-03
0.015771       11  0.004416                   11  3.007315e-03
0.022122       12  0.005696                   12  3.897294e-03
0.031031       13  0.007228                   13  4.961101e-03
0.043528       14  0.008826                   14  6.071824e-03
0.061057       15  0.010223                   15  7.064909e-03
0.085645       16  0.011554                   16  8.024002e-03
0.120136       17  0.012998                   17  9.059781e-03
0.168516       18  0.014568                   18  1.015998e-02
0.236378       19  0.016268                   19  1.131584e-02
0.331549       20  0.018108                   20  1.253636e-02
0.465100       21  0.019875                   21  1.381378e-02
0.652400       22  0.018135                   22  1.295566e-02
0.915100       23  0.012703                   23  9.352736e-03
1.283650       24  0.007916                   24  5.985086e-03
1.800600       25  0.004678                   25  3.612595e-03
2.525700       26  0.002548                   26  2.000073e-03
3.542800       27  0.001194                   27  9.314688e-04
4.969550       28  0.000454                   28  3.181398e-04
6.970850       29  0.000105                   29 -3.997808e-07
9.778100       30 -0.000093                   30 -2.009068e-04
13.715850      31 -0.000236                   31 -3.531055e-04
19.239350      32 -0.000339                   32 -4.653181e-04
26.987250      33 -0.000382                   33 -5.262790e-04
37.855300      34 -0.000383                   34 -5.476142e-04
53.100050      35 -0.000389                   35 -5.987324e-04
73.887500      36 -0.000414                   36 -6.579910e-04
97.662500      37 -0.000457                   37 -6.949428e-04
121.437500     38 -0.000525                   38 -7.261321e-04
145.212500     39 -0.000601                   39 -7.455744e-04
168.987500     40 -0.000671                   40 -7.608742e-04
192.762500     41 -0.000734                   41 -7.774773e-04
216.537500     42 -0.000793                   42 -8.113616e-04
240.312500     43 -0.000847                   43 -8.500524e-04
264.087500     44 -0.000899                   44 -8.825295e-04
287.862500     45 -0.000941                   45 -9.047741e-04
311.637500     46 -0.000936                   46 -8.851161e-04
335.412500     47 -0.000838                   47 -7.731427e-04
359.187500     48 -0.000653                   48 -5.879001e-04
382.962500     49 -0.000403                   49 -3.428049e-04
406.737500     50 -0.000089                   50 -3.095918e-05
430.512500     51  0.000291                   51  3.436834e-04
454.287500     52  0.000737                   52  7.827817e-04
478.062500     53  0.001250                   53  1.294337e-03
501.837500     54  0.001826                   54  1.876845e-03
525.612500     55  0.002461                   55  2.489562e-03
549.387500     56  0.003148                   56  3.159853e-03
573.162500     57  0.003875                   57  3.881572e-03
596.937500     58  0.004629                   58  4.644518e-03
620.712500     59  0.005396                   59  5.434263e-03
644.487500     60  0.006161                   60  6.205456e-03
668.262500     61  0.006910                  

# Fluxes by Layer

CRD                          CLIRAD             \
                        flug          fldg      fnetg      i       flug   
pressure    level                                                         
0.000000    1     -14.900415  0.000000e+00 -14.900415     11 -14.905418   
0.000624    2     -14.900416  5.979818e-08 -14.900415     11 -14.905418   
0.000876    3     -14.900416  8.631633e-08 -14.900415     11 -14.905418   
0.001229    4     -14.900416  1.314162e-07 -14.900415     11 -14.905418   
0.001723    5     -14.900416  2.081564e-07 -14.900415     11 -14.905418   
0.002417    6     -14.900416  3.383886e-07 -14.900415     11 -14.905418   
0.003391    7     -14.900416  5.588942e-07 -14.900415     11 -14.905418   
0.004757    8     -14.900416  9.313254e-07 -14.900415     11 -14.905418   
0.006672    9     -14.900416  1.558680e-06 -14.900415     11 -14.905418   
0.009359    10    -14.900417  2.613038e-06 -14.900414     11 -14.905419   
0.013128    11    -14.900417  4.381019e-06 -14.900412     11 -14.905419   
0.018415    12    -14.900417  7.339067e-06 -14.900410     11 -14.905419   
0.025830    13    -14.900417  1.227730e-05 -14.900405     11 -14.905419   
0.036232    14    -14.900416  2.048890e-05 -14.900396     11 -14.905419   
0.050823    15    -14.900414  3.383583e-05 -14.900380     11 -14.905418   
0.071291    16    -14.900410  5.473890e-05 -14.900356     11 -14.905416   
0.100000    17    -14.900403  8.690037e-05 -14.900316     11 -14.905412   
0.140271    18    -14.900391  1.362147e-04 -14.900254     11 -14.905405   
0.196760    19    -14.900368  2.115808e-04 -14.900157     11 -14.905392   
0.275997    20    -14.900331  3.262847e-04 -14.900004     11 -14.905370   
0.387100    21    -14.900266  4.999232e-04 -14.899766     11 -14.905332   
0.543100    22    -14.900158  7.590247e-04 -14.899399     11 -14.905270   
0.761700    23    -14.900016  1.087252e-03 -14.898929     11 -14.905191   
1.068500    24    -14.899887  1.419656e-03 -14.898467     11 -14.905130   
1.498800    25    -14.899792  1.727902e-03 -14.898064     11 -14.905105   
2.102400    26    -14.899738  2.008416e-03 -14.897729     11 -14.905123   
2.949000    27    -14.899733  2.259368e-03 -14.897474     11 -14.905192   
4.136600    28    -14.899787  2.481582e-03 -14.897306     11 -14.905323   
5.802500    29    -14.899907  2.691169e-03 -14.897216     11 -14.905525   
8.139200    30    -14.900105  2.918294e-03 -14.897187     11 -14.905817   
11.417000   31    -14.900410  3.186953e-03 -14.897223     11 -14.906233   
16.014700   32    -14.900868  3.516620e-03 -14.897352     11 -14.906820   
22.464000   33    -14.901550  3.939034e-03 -14.897610     11 -14.907647   
31.510500   34    -14.902573  4.552562e-03 -14.898020     11 -14.908838   
44.200100   35    -14.904143  5.547217e-03 -14.898596     11 -14.910582   
62.000000   36    -14.906575  7.158550e-03 -14.899417     11 -14.913321   
85.775000   37    -14.910170  9.587027e-03 -14.900583     11 -14.917459   
109.550000  38    -14.914094  1.222221e-02 -14.901871     11 -14.921928   
133.325000  39    -14.918279  1.493081e-02 -14.903349     11 -14.926570   
157.100000  40    -14.922682  1.763904e-02 -14.905043     11 -14.931241   
180.875000  41    -14.927277  2.034483e-02 -14.906932     11 -14.935932   
204.650000  42    -14.932048  2.304867e-02 -14.909000     11 -14.940664   
228.425000  43    -14.936983  2.575084e-02 -14.911232     11 -14.945547   
252.200000  44    -14.942069  2.845149e-02 -14.913618     11 -14.950610   
275.975000  45    -14.947301  3.115193e-02 -14.916149     11 -14.955816   
299.750000  46    -14.952685  3.388523e-02 -14.918800     11 -14.961151   
323.525000  47    -14.958326  3.689005e-02 -14.921436     11 -14.966709   
347.300000  48    -14.964348  4.055301e-02 -14.923795     11 -14.972528   
371.075000  49    -14.970761  4.512581e-02 -14.925635     11 -14.978656   
394.850000  50    -14.977499  5.072819e-02 -14.926771     11 -14.985079   
418.625000  51    -14.984488  5.746612e-02 -14.927

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')